# **1. Introduction to Spark**

Apache Spark is built for parallel data processing across machine clusters. It supports languages like Java, Python, R, and Scala. Spark includes libraries for a wide array of **tasks**, such as SQL, streaming, and machine learning. It's adaptable, running from a laptop to clusters of thousands of servers. This makes Spark beginner-friendly, yet it comes with a steep learning curve. Users can scale their work from small data processing **tasks** to handling big data at a very large scale.

The Apache Spark framework operates on a master-slave architecture, comprising two key components: a **driver program** and executors nodes. The term "nodes" refers to individual computers or servers that are part of a larger network or cluster. Each node is an independent unit with its own CPU, memory, and storage, capable of executing **tasks**. The framework utilizes a master-slave architecture to organize how these nodes work together. To illustrate, consider the following diagram for a higher overview of the architecture:

<center>
<img src="figures/architecture-spark.png">
</center>

- **Driver program:** This is the heart of the Spark application, running the main() function and creating the **SparkContext**. The **SparkContext** is essential for establishing communication with the **cluster manager** and creating Resilient Distributed Datasets (RDDs), which are distributed across the cluster and operated on in parallel. Also, Spark Driver contains various other components such as DAG Scheduler, Task Scheduler, Backend Scheduler, and Block Manager, which are responsible for translating the user-written code into jobs that are actually executed on the cluster.

- **Cluster Manager:** It's a service that can run on a node or across multiple nodes. The **cluster manager** is an external service for acquiring resources on the cluster (e.g., Apache Mesos, Hadoop YARN, or Spark's own standalone **cluster manager**). It allocates resources to Spark applications based on the requirements sent by the **Driver Program** through the **SparkContext**. It's responsible for allocating executor processes across the available worker nodes within the cluster as requested by the **Driver Program**.

- **Worker Node:** This is a node where Spark executors run **tasks**. Each **worker node** is a physical or virtual machine that is part of the Spark cluster. 
    
    - **Executors:** Are processes that run computations and store data for your application on the **worker node**.
    - **Tasks:** Tasks are individual units of work sent to the **executor** by the driver. Each task corresponds to a combination of data and computation.
    - **Cache:** This is used by executors to store data that can be reused in other **tasks**, reducing the need to fetch this data from disk or over the network for subsequent **tasks**.

The **SparkContext** in the **driver program** requests resources from the **cluster manager**, which then assigns **worker nodes**. Executors on the **worker nodes** then process **tasks** and store results. These results are sent back to the **driver program**, which may then initiate further **tasks** or return results to the user.

We can run Spark in three different modes: local mode, cluster mode, and client mode. The mode in which Spark runs determines the location of the **driver program** and the executors. The mode also affects how the **driver program** communicates with the **cluster manager** and the **worker nodes**. Consider each mode in more detail:

### **Cluster Mode**

The execution of a Spark application involves submitting the application to a **cluster manager** (such as Apache Mesos, Hadoop YARN, or Spark's own standalone **cluster manager**). Here, both the **driver program** and **executor** processes are launched within the cluster. Specifically, the driver process runs on one of the **worker nodes** designated by the **cluster manager**, separate from the machine where the submission occurred.

### **Client Mode**

Client mode operates similarly to cluster mode, with the key difference being the location of the Spark driver. In client mode, the driver runs on the client machine — the machine from which the application is submitted, often referred to as the gateway machine or edge node. This setup facilitates direct interaction between the application's user and the driver, making it easier to debug or monitor the application's progress. The **executor** processes, however, are still managed by the **cluster manager** and run within the cluster. Client mode is particularly useful during the development and debugging phases of an application, as it allows for immediate access to the driver's logs and prompts.

### **Local Mode**

Local mode simplifies the execution environment by running the entire Spark application on a single machine. Unlike cluster and client modes, which distribute **tasks** across multiple nodes in a cluster, local mode simulates a distributed environment using threads on one machine. This mode is ideal for development, testing, or experimentation, as it does not require a cluster and simplifies the setup process.


## **1.1 Batch Processing**

Batch processing in Spark executes jobs that handle large volumes of data in batches, contrasting with stream processing where data is processed continuously as it arrives. Each job processes a complete dataset at once.

The workflow of batch processing in Spark begins by reading data in large batches from diverse data sources. This data is then processed in parallel across the nodes in the cluster, utilizing Spark's core abstractions such as RDDs (Resilient Distributed Datasets), DataFrames, or Datasets, depending on the specific requirements of the task. The data goes to a series of transformations, orchestrated through Directed Acyclic Graph (DAG) operations, scheduling the job's execution across the cluster.

RDDs (Resilient Distributed Datasets), are a fundamental concept in Apache Spark, representing a distributed collection of objects. Internally, Spark DataFrames are built upon RDDs, introducing an additional layer of abstraction that facilitates more efficient data handling and processing. This is crucial for batch processing, where large datasets are divided into smaller partitions that are processed concurrently across multiple nodes, significantly speeding up processing times.

Spark's approach to batch processing involves constructing a DAG of RDD transformations for each job, optimizing the execution plan to minimize data shuffling and enhance job execution times. The scheduler divides the DAG into stages that can be executed concurrently, further optimizing resource usage and processing time for batch jobs.

The execution stages are divided into tasks, each designated for execution on an executor. Executors are basically JVM (java virtual machine) processes allocated to Spark applications by the cluster manager, running on worker nodes. The task scheduler efficiently distributes these tasks among the available executors, optimizing for data locality and balancing the workload to ensure the efficient execution of batch processing jobs.


# **2. Virtual Machine in Google Cloud**

The idea is to run a Spark application in a virtual machine in Google Cloud in local and cluster mode. The first step is to create a virtual machine in Google Cloud.Before configuring the Virtual machine, we need to generate a ssh key pair. This will allow us to connect to the VM instance using the ssh protocol. To generate the key pair, open a terminal go to the `.ssh` directory and run the following command:

```bash
ssh-keygen -t rsa -f ~/.ssh/KEY_FILENAME -C USERNAME -b 2048
```

Where `KEY_FILENAME` is the name of the file that will store the key pair and `USERNAME` is the username that will be used to connect to the VM instance. The `-b` flag is used to specify the number of bits in the key. The default value is 2048 bits. This created a private key file `KEY_FILENAME` and a public key file `KEY_FILENAME.pub`. The public key file will be used to configure the VM instance.

Now in the Google Cloud Plataform open the left bar and go to `Compute Engine > Metadata`, click on `SSH Keys` and `ADD SSH KEY` to add the public key to the list of SSH keys. The public key that need to be copied is the content of the `KEY_FILENAME.pub` file. After adding the key, click on `Save` to save the changes. At the end we should have the following:

<center>
<img src="figures/ssh-key.png">
</center>

Now any Virtual machine that is created will use this ssh key. This will allow us to connect to the VM instance using the private key. Now we in the Google Cloud Platform, go to the navigation bar on the left and select `Compute Engine > VM instances` to create a new VM instance. Click on `Create` to create a new VM instance. 

**Machine configuration**

The following image shows the configuration of the machine that will be used in this project.


<center>
<img src="figures/vm-setup.png">
</center>

**Boot Disk**

This will create a machine with 4 vCPUs and 16 GB of memory. The boot disk is a Ubuntu 22.04 LTS image with disk size of 40 GB. 

<center>
<img src="figures/boot-disk.png">
</center>

Then we can click on create to create the VM instance. We should see the following:

<center>
<img src="figures/vm-instance-ip.png">
</center>

Now copy the external ip (EXTERNAL_IP) address of the VM instance and go to the terminal to connect using the following command:

```bash
ssh -i ~/.ssh/KEY_FILENAME USERNAME@EXTERNAL_IP
```

The `-i` flag is used to specify the private key file. After running the command, we should be connected to the VM instance. In my specific case i should connect using `ssh -i ~/.ssh/gcp_key marcos@34.138.143.219`. We can check the machine configuration using the htop command. The following image shows the output of the htop command.

<center>
<img src="figures/htop-vm.png">
</center>

instead of using the entire ssh command, we can simplify by creating a `config` file in `.ssh` directory. The file should have the following content for each new ssh connection:

```bash
Host VM_NAME
  HostName EXTERNAL_IP
  User USERNAME
  IdentityFile ~/.ssh/KEY_FILENAME
```

For my particular case, to connect to google Cloud, this would be:

```bash
Host de-bootcamp
  HostName 34.138.143.219
  User marcos
  IdentityFile ~/.ssh/gcp_key
```

then we can connect to the VM instance simply using:

```bash
ssh VM_NAME
```

We can also configure our VScode to connect via ssh to the VM instance. To do this, we need to install the `Remote - SSH` extension. After installing the extension, click on the green icon on the bottom left of the VScode and select `Open a Remote Window:Connect to Host...`. After selecting the correct connection, the VScode will connect to the VM instance and we can start coding in the VM instance. For more details check [this](https://code.visualstudio.com/docs/remote/ssh). We should have something like:

<center>
<img src="figures/vscode-ssh.png">
</center>

## **2.1 Spark Installation**

To install Spark on the Virtual Machine is similar to the installation in a local computer with ubuntu via terminal. We first install Java and then Spark and Pyspark.

### **java Installion**

To install java, download OpenJDK 11 in [OpenJDK](https://jdk.java.net/archive/) and download it to `~/spark` folder and Unpack it with:

```bash
wget https://download.java.net/java/GA/jdk11/9/GPL/openjdk-11.0.2_linux-x64_bin.tar.gz

tar xzfv openjdk-11.0.2_linux-x64_bin.tar.gz
```
the flag `xzfv` means to extract, decompress the archive using gzip before extracting, indicate that the next argument is the name of the file and print the names of files as they are extracted, respectively. In the `.bashrc` we need to define `JAVA_HOME` and add it to `PATH` to make it available to the system when the terminal are opened. Go to the `~/.bashrc` file and add the following:

```bash
export JAVA_HOME="${HOME}/spark/jdk-11.0.2"
export PATH="${JAVA_HOME}/bin:${PATH}"
```

check that it works:

```bash
java --version
```

### **Spark Installation**


To install Spark, download [Spark](https://spark.apache.org/downloads.html) to the same folder `~/spark` and unpack it with:

```bash
wget https://dlcdn.apache.org/spark/spark-3.4.2/spark-3.4.2-bin-hadoop3-scala2.13.tgz

tar xzfv spark-3.4.2-bin-hadoop3-scala2.13.tgz
```

Again, in the `.bashrc` add a new path for spark to `PATH`:

```bash
export SPARK_HOME="${HOME}/spark/spark-3.4.2-bin-hadoop3-scala2.13"
export PATH="${SPARK_HOME}/bin:${PATH}"
```

To check if it works, open a new terminal, execute `spark-shell` and run the following:

```bash
val data = 1 to 10000
val distData = sc.parallelize(data)
distData.filter(_ < 10).collect()
```

<center>
<img src="data/spark-test.png" >
</center>

### **Pyspark Installation**

Assuming that we already have python on the VM to run PySpark, we need to add it to `PYTHONPATH`:

```bash
export PYTHONPATH="${SPARK_HOME}/python/:$PYTHONPATH"
export PYTHONPATH="${SPARK_HOME}/python/lib/py4j-0.10.9.7-src.zip:$PYTHONPATH"
```
This tells the Python interpreter where to locate additional modules and packages that are not part of the standard library or not in the current directory. Make sure that the version under `${SPARK_HOME}/python/lib/` matches the filename of py4j or will encounter `ModuleNotFoundError: No module named 'py4j'` while executing `import pyspark`.

Another simple way is use pip to install the package `pyspark`:

```bash
pip install pyspark
```

This will automatically add the path to `PYTHONPATH`.


### **Pyspark Installation**

Assuming that we already have python on the VM to run PySpark, we need to add it to `PYTHONPATH`:

```bash
export PYTHONPATH="${SPARK_HOME}/python/:$PYTHONPATH"
export PYTHONPATH="${SPARK_HOME}/python/lib/py4j-0.10.9.7-src.zip:$PYTHONPATH"
```
This tells the Python interpreter where to locate additional modules and packages that are not part of the standard library or not in the current directory. Make sure that the version under `${SPARK_HOME}/python/lib/` matches the filename of py4j or will encounter `ModuleNotFoundError: No module named 'py4j'` while executing `import pyspark`.

Another simple way is use pip to install the package `pyspark`:

```bash
pip install pyspark
```

This will automatically add the path to `PYTHONPATH`.



## 2.2 Testing

When setting up a development environment with VSCode to work with Jupyter notebooks, we might encounter a situation where VSCode does not automatically recognize environment variables such as JAVA_HOME and SPARK_HOME. In this case, if we are running in our local machine, we can open VScode via terminal by using `code`. This will allow VSCode to recognize the environment variables. But, if we are running in a VM instance from Google Cloud, for example, and connecting the VScode by the Remote-SSH application, a practical solution involves using port forwarding to access the Jupyter notebook through a web browser. Here I add port 8888 as show below:

<center>
<img src="figures/port.png">
</center>

after this, make sure to have jupyter installed in the VM instance. To open the jupyter notebook, go to the terminal, connect with the VM instance and run the following:

```bash
jupyter notebook
```
We should have something like:

<center>
<img src="figures/run-jupyter-terminal.png">
</center>

This will be running in port `http://localhost:8888/`. The server provides a URL that includes a token-based authentication part, which looks like `?token=<some_long_string>`. This token is a security measure to prevent unauthorized access to our Jupyter notebooks, and in this case the HTTP request address is `http://localhost:8888/?token=f4c5f14db0b2934c18e2a911c85270067f6f7ba79f677510`.

Now download the Taxi Zone Lookup CSV file from [NYC taxi](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page) and save inside the `data` directory :

```bash
wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
```

We can check this data with Pyspark by running the follow:

In [10]:
import pyspark
from pyspark.sql import SparkSession

TThe following code initiates a `SparkSession`, an entry point for programming Spark with the Dataset and DataFrame API. It is used to create DataFrames, register DataFrames as tables, execute SQL over tables, cache tables, and read parquet files. As we have seen before, Spark has three modes: cluster, client, and local. For testing, let's use the local mode by setting `.master("local[*]")`. This means that Spark will run locally on the virtual machine instance from Google Cloud, using as many worker threads as there are logical cores on the machine. The method `.appName("test")` sets the name of the application to be shown in the Spark web UI, and the method `.getOrCreate()` creates the SparkSession with all the parameters set previously.

In [11]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

# Suppress warnings
spark.sparkContext.setLogLevel("ERROR")


In [12]:
# Read the taxi zone lookup and show
# Set header to true to consider first row as header
df = spark.read \
    .option("header", "true") \
    .csv('data/taxi+_zone_lookup.csv')

df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

When we run the `df.write.parquet('zones')` command in PySpark, the resulting `zones` directory will contains several files due to how Spark manages data storage in Parquet format. The `_SUCCESS` file is a marker indicating the write operation completed successfully. Accompanying CRC (Cyclic Redundancy Check) files, like `._SUCCESS.crc`, provide cyclic redundancy checks for data integrity verification. The data itself is stored in `.snappy.parquet` files, with their names including a unique identifier and a partition index, indicating they contain a portion of the DataFrame's data compressed using Snappy for efficiency. Each `.snappy.parquet` file also has a corresponding `.crc` file for integrity checks. The presence of these files reflects Spark's distributed data processing approach, where data can be partitioned across multiple files for scalable processing and storage.

In [13]:
# writes the data in df to the zones directory in Parquet format
df.write.parquet('zones')

# **3. Spark DataFrames**

# **4. Group By**

# **5. Joins in Spark**

# **6. Creating a local Spark Cluster**

# **7. Setting up a Dataproc Cluster**